<div style="float:left">
    <h1 style="width:600px">CASA0006 Practical 4: Analysis workflow</h1>
    <h3 style="width:600px">CASA0006: Data Science for Spatial Systems</h3>
</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

In [ ]:
import datetime
now = datetime.datetime.now()
print("Last executed: " + now.strftime("%Y-%m-%d %H:%M:%S"))

## Welcome!

In this workshop, we will achieve the following objectives:
1. Review weekly quiz;
1. Build a sklearn **pipeline** to predict the daily bicycle rentals using four different methods, including linear regression, CART, random forest, and XGBoost;
1. Diagnose the four models by comparing the training and testing error;
1. Compute the bias and variance of the machine learning models using the **mlxtend** library.

In the following code, some codes are missing and marked with ```??```.

Please replace ```??``` with the correct codes, using the hints and context. The solution will be given in the workshop.

\*\* Review the weekly quiz \*\*.

First, import the relevant libraries. 

* `pandas` for data import and handling;
* `matplotlib`;
* `numpy`;
* `sklearn`;
* `xgboost` for XGBoost models.

**Run the script below to get started.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve
from sklearn.metrics import root_mean_squared_error, r2_score

# preprocessors
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# pipeline
from sklearn.pipeline import Pipeline

# linear regression
from sklearn.linear_model import LinearRegression

# CART
from sklearn.tree import DecisionTreeRegressor

# random forest
from sklearn.ensemble import RandomForestRegressor

# xgboost
import xgboost
from xgboost import XGBRegressor

pd.set_option('display.max_rows', 300) # specifies number of rows to show
pd.options.display.float_format = '{:40,.4f}'.format # specifies default number format to 4 decimal places
plt.style.use('ggplot') # specifies that graphs should use ggplot styling
%matplotlib inline

In [ ]:
# check the library version before we start
print("xgboost version:{}".format(xgboost.__version__))
print("sklearn version:{}".format(sklearn.__version__))

## Data Loading and Exploration

We will use the same bicyle rental data as Week 3 workshop. 

The dataset relates to daily counts of rented bicycles from the bicycle rental company Capital-Bikeshare in Washington D.C., along with weather and seasonal information. The goal here is to predict how many bikes will be rented depending on the weather and the day. 

The original data can be downloaded from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset).

The dataset used in this workshop has been slightly processed by Christoph Molnar using the processing R-script from this [Github repository](https://github.com/christophM/interpretable-ml-book/blob/master/R/get-bike-sharing-dataset.R). Here, the dataset is provided as a csv file on Moodle.

Here is a list of the variables in the dataset:

- Count of bicycles including both casual and registered users. The count is used as the response in the regression task.
- Indicator of the season, either spring, summer, fall or winter.
- Indicator whether the day was a holiday or not.
- The year: either 2011 or 2012.
- Number of days since the 01.01.2011 (the first day in the dataset). This predictor was introduced to take account of the trend over time.
- Indicator whether the day was a working day or weekend.
- The weather situation on that day. One of:
  - **'GOOD'**: including clear, few clouds, partly cloudy, cloudy
  - **'MISTY'**: including mist + clouds, mist + broken clouds, mist + few clouds, mist
  - **'RAIN/SNOW/STORM'**: including light snow, light rain + thunderstorm + scattered clouds, light rain + scattered clouds, heavy rain + ice pallets + thunderstorm + mist, snow + mist
- Temperature in degrees Celsius.
- Relative humidity in percent (0 to 100).
- Wind speed in km/h.

We will use Pandas package to load and explore this dataset:

- Import the Boston housing dataset as a Pandas dataframe (call it `bike_rental`)
- Inspect the data
- Calculate summary statistics on all attributes

In [ ]:
bike_rental = pd.read_csv('https://raw.githubusercontent.com/huanfachen/Spatial_Data_Science/main/Dataset/daily_count_bike_rental.csv')
# drop the year variable as it is not useful
bike_rental = bike_rental.drop(['yr'], axis=1)

In [ ]:
bike_rental.??()

In `bike_rental`, there are two data types: categorical (aka `object`), and numerical (including `int64` and `float64`). 

In [ ]:
# print a few rows of this dataset
bike_rental.head()

Is there missing value in each column?

In [ ]:
bike_rental.??().sum()

## Building a pipeline

The workflow of the week-3 workshop is as follows:

1. One-hot encoding of the categorical variables, including seasons, month, holiday, weekday, workingday, weathersit;
1. Split the data into training and testing subsets;
1. Train the model on the training subset (including hyperparameter tuning);
1. Report the model performance using the testing subsets;
1. Repeat the above two steps and compare the performance of different models.

In week-3 workshop, we conducted the above steps separately and step-by-step. This approach has two potential problems:

1. The readers may find it difficult to get the full picture of analysis;
2. It may lead to data leakage issues.

Here, we will use a similar workflow but using a **pipeline** to consolidate these steps.

In the folllowing, we are going to:

1. Split the data into training and testing subsets (note: we still need to manually split the training and testing subsets before using a pipeline);
1. Build a pipeline that consists of one-hot encoding, model training, and model evaluation.

In [ ]:
random_state_split = 100
train_x, test_x, train_y, test_y = train_test_split(bike_rental.drop(['cnt'], axis = 1), bike_rental.cnt, random_state=random_state_split)

What is a **pipeline** in sklearn? The purpose of the pipeline is to assemble several steps of data processing and model training while setting different parameters.

From the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), the main parameter of a pipeline is 'steps':

```
steps: list of tuple
List of (name, transform) tuples (implementing fit/transform) that are chained in sequential order. The last transform must be an estimator.
```

Let's make it easier: a pipeline should look like this:

```
Pipeline(steps=[('name_of_preprocessor', preprocessor),
                ('name_of_ml_model', ml_model())])
```

Or, like this graph:

![](https://github.com/huanfachen/DSSS/blob/main/Figures/pipeline.png?raw=true)


### The preprocessor of a pipeline

A preprocessor of a pipeline are a list of tranforming or data processing steps.

Firstly, we need to define the transformers for both numeric and categorical features. 

A transforming step is represented by a tuple. In that tuple, you first define the name of the transformer, and then the function you want to apply. The order of the tuple will be the order that the pipeline applies the transforms. 

Here, we first deal with missing values, then standardise numeric features and one-hot encode categorical features.

*Why dealing with missing values?* Although there are no missing values in this dataset, we demonstrate this transformer, as we are likely to deal with missing values in other tasks.

*Why standardising numeric features?* While some algorithms don't require data standardisation (e.g. linear regression, decision trees), standardising numeric features doesn't degrade the model performance and other algorithms prefer standardised input data, such as neural networks.

*Why one-hot encoding categorical features?* This is becuase many ML algorithms can't deal with categorical features.

In [ ]:
# The missing values of a numeric feature will be replaced by the 'mean' value of the feature.
# The missing values of a categorical feature will be replaced by the a 'constant' value. If 'constant' value is not specified, it is default at '0' for numerical data or 'np.nan' for categorical data.

numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OneHotEncoder(drop='first'))
])

Next, we need to specify which columns are numeric and which are categorical, so the pipeline can apply the transformers accordingly.

 We apply the transformers to features by using **ColumnTransformer**.

 From the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), a ColumnTransformer applies transformers to columns of an array or pandas DataFrame.

The main parameter is *transformers*:

```
transformers: list of tuples
List of (name, transformer, columns) tuples specifying the transformer objects to be applied to subsets of the data.
```
 
 Similar to pipeline, we pass a list of tuples, which is composed of (‘name’, ‘transformer’, ‘features’), to the parameter ‘transformers’.

In [ ]:
numeric_features = ['temp', 'hum', 'windspeed', 'days_since_2011']
categorical_features = ['season', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', ??, ??)
])

### Adding an estimator to a pipeline

With the preprocessor assembled, we can add in the estimator, which is any machine learning algorithm we would like to apply, to complete our preprocessing and training pipeline. 

As this is a regression task, we will start with a decision tree regressor.

In [ ]:
pipeline = Pipeline(steps = [
   ('preprocessor', preprocessor),
   ('regressor',DecisionTreeRegressor())
])

In [ ]:
cart_model = pipeline.fit(??, ??)
# this will visualise the pipeline
print(cart_model)

Then, we can evaluate the model using testing data.

In [ ]:
print("RMSE on the training data:")
print(root_mean_squared_error(train_y, cart_model.predict(train_x)))
print("RMSE on the testing data:")
print(root_mean_squared_error(test_y, cart_model.predict(test_x)))

In [ ]:
print("R2 on the training data:")
print(r2_score(train_y, cart_model.predict(train_x)))
print("R2 on the testing data:")
print(r2_score(test_y, ??.predict(test_x)))

## Using pipeline for hyperparameter tuning

The pipeline can be used for hyperparameter tuning. We will demonstrate how to find the optimal max_depth and min_samples_split using pipeline and GridSearchCV.

From the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) of GridSearchCV, the first parameter *estimator* is:

```
estimator: estimator object
This is assumed to implement the scikit-learn estimator interface. Either estimator needs to provide a score function, or scoring must be passed.
```

As a pipeline is similar to a model instance and is an estimator, we can combine pipeline with GridSearchCV for tuning hyperparameters.

One trick here: when specifying the grid_params, the name of each hyperparameter should be in the format of ```[estimator]__[hyperparameter]```, where the first part of ```[estimator]``` is the name of estimator in the Pipeline object, the second part is **two underscores**, and the third part is the hyperparameter in the model.

For example: ```regressor__max_depth```.

See more details [here](https://stackoverflow.com/a/41899244/4667468).

In [ ]:
# we fix the random_state in DecisionTreeRegressor() so that the result of GridSearchCV is the same in different runs
cart_pipeline = Pipeline(steps = [
  ('preprocessor', preprocessor),
  ('regressor', DecisionTreeRegressor(random_state=123))
])

cart_pipeline.fit(train_x, train_y)

# grid_params is the range of each hyperparameter
grid_params = {
  'regressor__max_depth': [10,20,30,40,40], 
  '??min_samples_split': [2,4,6,8,10]
}
search = GridSearchCV(cart_pipeline, grid_params)
search.fit(train_x, train_y)
print("Best R2 Score: ", search.best_score_)
print("Best Params: ", search.best_params_)

## Comparing several estimators

Scikit learn pipelines make the workflows smoother and more flexible. With the pipeline, we can easily compare the performance of a number of algorithms:

In [ ]:
regressors = {
    'Linear': LinearRegression(),
    'CART': DecisionTreeRegressor(),
    'RF': ??(),
    'XGB': ??()
}

# a dict to store the R2 of training and testing data
dict_results = dict()

for name, regressor in regressors.items():
    pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor', regressor)
           ])
    model = pipeline.fit(train_x, train_y)
    predictions = model.predict(test_x)
    dict_results[name] = [model.score(train_x, train_y), model.score(test_x, test_y), model.score(??, ??) - model.score(??, ??)]

# transform dict_models to dataframe
df_models = pd.DataFrame.from_dict(dict_results, orient='index', columns=['R2_train_data', 'R2_test_data', 'R2_diff'])
df_models

We can compare the four models using their R2 on the training and testing data. 

Note the different direction of R2 and prediction errors (e.g. RMSE): a large R2 means a small predictive error.

Regarding the training R2, both CART and XGBoost have the largest R2, followed by random forest and then linear model. For the different between training R2 and testing R2, the linear model has the least value, while CART has the largest difference between training and testing R2.

Some initial conclusions on the model performance:

1. The linear model is underfitting, as it has a relatively low R2 on the training data.
2. The CART may be subject to overfitting, as it has a relatively low R2 on the testing data. 
3. Compared with the CART, the random forest and XGBoost model are less subject to overfitting, as theire R2 score on the testing data is higher than CART. These two models have better performance than linear model or CART.

## Calculating the model bias and variance [Optional]

In this part, we will demonstrate how to compute the bias and variance of a ML model. 

This library has been developed by [Sebastian Raschka](https://sebastianraschka.com/) and it provides a function named bias_variance_decomp() that help us to estimate the bias vs variance for various models over many bootstrap samples. 

The basic idea of bias_variance_decomp() is that: given the training and testing data and the predictive algorithm, in each round, it get a bootstrap sample of the training data, which is used to train a model, and then the model is applied to the testing data, which leads to predictions on the testing data. 

As the sampled training data is different in each round, the predictions on the testing data differ in each round. We can combine the multilple-round predictions (on the testing data) to estimate the bias and variance of the predictive algorithm.

- The bias is the difference between the average prediction across multiple rounds and the actual values (of the testing data)

- The variance is the variance of predictions over multiple rounds (of the testing data)

Please note - the bias and variance of the algorithm is evaluated using the multiple-round prediction errors on the testing data, rather than the training data. The training data is only used to train the model, instead of evaluating the model.

You can read the documentation [here](https://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/) and code [here](https://github.com/rasbt/mlxtend/blob/master/mlxtend/evaluate/bias_variance_decomp.py).

To do this, we first need to install a new library called 'mlxtend'.

In [ ]:
!pip install mlxtend==0.21.0

Import the bias_variance_decomp function.

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

First, we will calculate the bias and variance of the CART.

Note that here is a conflict between sklearn pipeline and bias_variance_decomp: a pipeline requires a Pandas DataFrame as input, while bias_variance_decomp requires a numpy array.

In order to use bias_variance_decomp(), we have to go back to the approach in week-3 workshop: 

1. one-hot encoding the categorical variables;
1. do the train-test split;
1. transform training and testing data into numpy arrays, using the DataFrame.to_numpy() method, see [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy)
1. create and train a DecisionTreeRegressor model.

In [ ]:
# one-hot encoding
bike_rental_numeric = pd.get_dummies(bike_rental)
bike_rental_final = bike_rental_numeric.drop(['season_SPRING', 'mnth_JAN', 'holiday_NO HOLIDAY', 'weekday_MON', 'workingday_WORKING DAY', 'weathersit_GOOD'], axis=??)

# train-test split
random_state_split = 100
train_x, test_x, train_y, test_y = train_test_split(bike_rental_final.drop(['cnt'], axis = 1), bike_rental_final.cnt, random_state=random_state_split)

Several key parameters of this function:

```
- loss : str (default='0-1_loss'). Loss function for performing the bias-variance decomposition. Currently allowed values are '0-1_loss' (for classification tasks) and 'mse' (Mean Square Error, for regression tasks).
- num_rounds : int (default=200). Number of bootstrap rounds (sampling from the training set) for performing the bias-variance decomposition. Each bootstrap sample has the same size as the original training set.
```

In [ ]:
# create and train a DecisionTreeRegressor model using numpy array datasets. compute the total error, bias, and variance of this model
avg_expected_loss, avg_bias_squared, avg_var = bias_variance_decomp(
        DecisionTreeRegressor(random_state=0), train_x.to_numpy(), train_y.to_numpy(), test_x.to_numpy(), test_y.to_numpy(), 
        loss='mse',
        random_seed=123)

print('Average expected loss: %.3f' % ??)
print('Average bias squared: %.3f' % ??)
print('Average variance: %.3f' % ??)

From the result above, we can see that expected loss is the sum of bias and variance. 

*Note:* In the [documentation of bias_variance_decomp](https://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/), there are some typos. It should be “bias squared” rather than “Bias”, as **Loss = bias^2 + variance**. This has been confirmed by the authors in [this issue](https://github.com/rasbt/mlxtend/issues/1083).

Now, we can compute and compare the bias and variance of the four models. 

To reduce the computing time, we set the parameter of num_rounds to 40 (default=200).

The following code takes 2 minutes on my desktop.

In [ ]:
random_seed = 1233
regressors = {
    'Linear': LinearRegression(),
    'CART': DecisionTreeRegressor(random_state = random_seed),
    'RF': RandomForestRegressor(random_state = random_seed),
    'XGB': XGBRegressor(random_state = random_seed)
}

# a dict to store the R2 of training and testing data
dict_results = dict()

for name, regressor in regressors.items():
    avg_expected_loss, avg_bias_squared, avg_var = bias_variance_decomp(
        regressor, train_x.to_numpy(), train_y.to_numpy(), test_x.to_numpy(), test_y.to_numpy(), 
        loss='mse',
        random_seed=123,
        num_rounds=40)
    dict_results[name] = [avg_expected_loss, avg_bias_squared, avg_var]

# transform dict_models to dataframe
df_models = pd.DataFrame.from_dict(dict_results, orient='index', columns=['Total loss', 'Bias^2', 'Variance'])
df_models

The above result shows that the linear model has the highest bias and lowest variance. Clearly, it is under-fitting.

Compared with CART, RF substantially decreases the variance and slightly decreases the bias, while XGBoost substantially decreases both bias and variance. Therefore, the RF and XGBoost model have the best performance for predicting the daily bicycle rental.

## Summary

In this workshop, we have created and used the machine learning pipelines for predicting the daily bicycle rental.

We conducted model diagnosis of the four methods via compared the predictive accuracy on the training and testing data.

As an optional task, we computed the bias and variance of these four methods using the mlxtend library.

## References and recommendations:

1. [Step by Step Tutorial of Sci-kit Learn Pipeline](https://towardsdatascience.com/step-by-step-tutorial-of-sci-kit-learn-pipeline-62402d4629b6): this post demonstrates the sklearn pipeline using a very similar dataset, but I disagree with using OrdinalEncoder for categorical variables.
2. [Tutorials on the bias_variance_decomp() function](http://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/): this is a handy tutorial from the author of the mlxtend library.